<a href="https://colab.research.google.com/github/huydt84/Polyp_segment/blob/main/polyp_segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile

local_zip = '/tmp/Kvasir_SEG_Training_880.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

local_zip = '/tmp/Kvasir_SEG_Validation_120.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

In [ ]:
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

def load_data_aug(path):
     images = sorted(glob(os.path.join(path, "image/*")))     
     masks = sorted(glob(os.path.join(path, "mask/*")))
     return images, masks

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def augment_data(images, masks, save_path, augment=True):
    H = 256
    W = 256

    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        """ Augmentation """
        if augment == True:
            aug = CenterCrop(H, W, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            save_images = [x, x1, x2, x3, x4, x5]
            save_masks =  [y, y1, y2, y3, y4, y5]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}.{image_extn}"
                tmp_mask_name = f"{mask_name}.{mask_extn}"

            else:
                tmp_img_name = f"{image_name}_{idx}.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}.{mask_extn}"

            image_path = os.path.join(save_path, "image", tmp_img_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

path = "/tmp/Kvasir_SEG_Training_880/"
images, masks = load_data_aug(path)
create_dir("/tmp/new_data/image")
create_dir("/tmp/new_data/mask")
augment_data(images, masks, "/tmp/new_data", augment=True)

100%|██████████| 880/880 [00:45<00:00, 19.43it/s]


In [ ]:
import numpy as np
#import cv2
#from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
#from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

def load_data(split=0.1):
    images = sorted(glob(os.path.join('/tmp/Kvasir_SEG_Training_880/image/*'))) 
    images_aug = sorted(glob(os.path.join('/tmp/new_data/image/*')))
    masks = sorted(glob(os.path.join('/tmp/Kvasir_SEG_Training_880/mask/*'))) 
    masks_aug = sorted(glob(os.path.join('/tmp/new_data/mask/*')))

    print(len(images), len(images_aug))
    images += images_aug
    masks += masks_aug

    test_x = sorted(glob(os.path.join('/tmp/Kvasir_SEG_Validation_120/images/*')))
    test_y = sorted(glob(os.path.join('/tmp/Kvasir_SEG_Validation_120/masks/*')))

    total_size = len(images)
    valid_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)



def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_model(num_filters = [16, 32, 48, 64]):
    size = 256
    #num_filters = [16, 32, 48, 64]
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs
    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()
    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

In [ ]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
## Dataset
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()

## Hyperparameters
batch = 8
lr = 1e-4
epochs = 14

train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

model = build_model()

opt = tf.keras.optimizers.Adam(lr)
metrics = ['acc', tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1

print(len(train_x))
model.fit(train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps)

880 5280
5544
Epoch 1/14
693/693 [==============================] - 157s 214ms/step - loss: 0.5230 - acc: 0.7499 - recall_1: 0.7621 - precision_1: 0.4247 - iou: 0.3191 - val_loss: 0.4776 - val_acc: 0.8245 - val_recall_1: 0.6898 - val_precision_1: 0.5345 - val_iou: 0.3593
Epoch 2/14
693/693 [==============================] - 147s 213ms/step - loss: 0.4375 - acc: 0.8198 - recall_1: 0.7457 - precision_1: 0.5338 - iou: 0.3971 - val_loss: 0.4066 - val_acc: 0.8695 - val_recall_1: 0.6528 - val_precision_1: 0.6663 - val_iou: 0.4294
Epoch 3/14
693/693 [==============================] - 148s 213ms/step - loss: 0.3780 - acc: 0.8527 - recall_1: 0.7351 - precision_1: 0.6094 - iou: 0.4574 - val_loss: 0.3586 - val_acc: 0.8903 - val_recall_1: 0.6270 - val_precision_1: 0.7674 - val_iou: 0.4806
Epoch 4/14
693/693 [==============================] - 149s 214ms/step - loss: 0.3295 - acc: 0.8743 - recall_1: 0.7308 - precision_1: 0.6731 - iou: 0.5106 - val_loss: 0.3335 - val_acc: 0.8937 - val_recall_1: 0.632

In [ ]:
test_dataset = tf_dataset(test_x, test_y, batch=batch)
model.evaluate(test_dataset, steps=len(test_x)//batch)

15/15 [==============================] - 2s 108ms/step - loss: 0.2894 - acc: 0.9131 - recall_1: 0.6024 - precision_1: 0.8337 - iou: 0.5619


[0.2893541753292084,
 0.9131134152412415,
 0.6024466753005981,
 0.8337039947509766,
 0.5618537068367004]

In [ ]:
model2 = build_model([16, 32, 64, 128])
model2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_57 (Conv2D)             (None, 256, 256, 16  448         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 256, 256, 16  64         ['conv2d_57[0][0]']              
 ormalization)                  )                                                           

In [ ]:
model2.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

In [ ]:
model2.fit(train_dataset,
    validation_data=valid_dataset,
    epochs=5,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps)

Epoch 1/5
693/693 [==============================] - 161s 233ms/step - loss: 0.1454 - acc: 0.9384 - recall_1: 0.8174 - precision_1: 0.8838 - iou: 0.7512 - val_loss: 0.2055 - val_acc: 0.9263 - val_recall_1: 0.6965 - val_precision_1: 0.9135 - val_iou: 0.6680
Epoch 2/5
693/693 [==============================] - 162s 233ms/step - loss: 0.1330 - acc: 0.9424 - recall_1: 0.8301 - precision_1: 0.8938 - iou: 0.7698 - val_loss: 0.1834 - val_acc: 0.9312 - val_recall_1: 0.7527 - val_precision_1: 0.8901 - val_iou: 0.6974
Epoch 3/5
693/693 [==============================] - 162s 234ms/step - loss: 0.1265 - acc: 0.9447 - recall_1: 0.8349 - precision_1: 0.9012 - iou: 0.7796 - val_loss: 0.2005 - val_acc: 0.9250 - val_recall_1: 0.7522 - val_precision_1: 0.8580 - val_iou: 0.6731
Epoch 4/5
693/693 [==============================] - 162s 233ms/step - loss: 0.1172 - acc: 0.9478 - recall_1: 0.8425 - precision_1: 0.9107 - iou: 0.7940 - val_loss: 0.1975 - val_acc: 0.9282 - val_recall_1: 0.7219 - val_precision_

In [ ]:
model2.evaluate(test_dataset, steps=len(test_x)//batch)

15/15 [==============================] - 2s 111ms/step - loss: 0.2450 - acc: 0.9183 - recall_1: 0.7285 - precision_1: 0.7817 - iou: 0.6141


[0.2450370192527771,
 0.9183209538459778,
 0.7284885048866272,
 0.7817497849464417,
 0.6141158938407898]

In [ ]:
model2.save("/tmp/model2_15_256_aug.h5")